In [8]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import torch
import skorch
import scipy
import torch.nn as nn
import torch
import torch.nn.functional as F
import sys
from skorch.helper import DataFrameTransformer
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn import metrics
from sklearn.preprocessing import FunctionTransformer
from skorch.callbacks import EarlyStopping
from sklearn.pipeline import Pipeline
import pickle
def identity_tokenizer(text):
    return text

In [6]:
with open("x_train.txt", "rb") as fp:   # Unpickling
    x_train = pickle.load(fp)
with open("y_train.txt", "rb") as fp:   # Unpickling
    y_train = pickle.load(fp)
with open("x_test.txt", "rb") as fp:   # Unpickling
    x_test = pickle.load(fp)
with open("y_test.txt", "rb") as fp:   # Unpickling
    y_test = pickle.load(fp)

### Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()


def typechange(x):
    return x.astype(dtype = np.float32)
typetransform = FunctionTransformer(typechange)

pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("log_model", log_model)])


pipe.fit(X=x_train, y=y_train)
y_pred = pipe.predict(x_test)
rmse = mean_squared_error(y_test, y_pred, squared = False)
print('RMSE:', rmse)

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


ValueError: Unknown label type: 'continuous'

### Multiple Regression

In [10]:
from sklearn.linear_model import LinearRegression
lin_model = LinearRegression()


def typechange(x):
    return x.astype(dtype = np.float32)
typetransform = FunctionTransformer(typechange)

pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("lin_model", lin_model)])


pipe.fit(X=x_train, y=y_train)
y_pred = pipe.predict(x_test)
rmse = mean_squared_error(y_test, y_pred, squared = False)
print('RMSE:', rmse)

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


RMSE: 2.061070014836731


### Ridge Regression

In [11]:
from sklearn.linear_model import Ridge
ridge_model = Ridge()


def typechange(x):
    return x.astype(dtype = np.float32)
typetransform = FunctionTransformer(typechange)

pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("ridge_model", ridge_model)])


pipe.fit(X=x_train, y=y_train)
y_pred = pipe.predict(x_test)
rmse = mean_squared_error(y_test, y_pred, squared = False)
print('RMSE:', rmse)

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


RMSE: 1.9897982


### Lasso

In [13]:
from sklearn.linear_model import Lasso
lasso_model = Lasso()


def typechange(x):
    return x.astype(dtype = np.float32)
typetransform = FunctionTransformer(typechange)

pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("lasso_model", lasso_model)])


pipe.fit(X=x_train, y=y_train)
y_pred = pipe.predict(x_test)
rmse = mean_squared_error(y_test, y_pred, squared = False)
print('RMSE:', rmse)

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


RMSE: 2.0521414


### Elastic Net

In [15]:
from sklearn.linear_model import ElasticNet
ElasticNet_model = ElasticNet()


def typechange(x):
    return x.astype(dtype = np.float32)
typetransform = FunctionTransformer(typechange)

pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("ElasticNet_model", ElasticNet_model)])


pipe.fit(X=x_train, y=y_train)
y_pred = pipe.predict(x_test)
rmse = mean_squared_error(y_test, y_pred, squared = False)
print('RMSE:', rmse)

RMSE: 2.0521414
